In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os
from tqdm import tqdm # 진행 상황 바를 표시하기 위한 라이브러리

# .env 파일에서 환경 변수 로드 (예: OpenAI API 키)
load_dotenv()

# --- 설정 (Configuration) ---
# OpenAI 임베딩 모델 설정
# "text-embedding-3-large"는 고품질의 임베딩을 제공합니다.
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 텍스트 분할기 설정
# chunk_size: 각 텍스트 청크의 최대 문자 수
# chunk_overlap: 인접한 청크 간의 중복 문자 수 (컨텍스트 유지에 도움)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 벡터 DB 저장 경로 및 대상 디렉토리 설정
# name: 각 DB의 식별 이름
# path: 원본 PDF 파일이 있는 디렉토리 경로 (하위 디렉토리 포함)
# persist_path: 생성될 Chroma 벡터 DB가 저장될 경로
targets = [
    {"name": "DB1", "path": r"C:\skn13\final\DB1", "persist_path": "./vector_db/db1_regulation"},
    {"name": "DB2", "path": r"C:\skn13\final\DB2", "persist_path": "./vector_db/db2_internal"}
]

# 벡터 삽입 시 사용할 배치(Batch) 크기
# 너무 크면 메모리 문제가 발생할 수 있고, 너무 작으면 시간이 오래 걸릴 수 있습니다.
# 50~200 사이에서 조절하는 것이 일반적입니다.
BATCH_SIZE = 100 

# --- 주 처리 로직 (Main Processing Logic) ---
for target in targets:
    print(f"\n--- {target['name']} DB 처리 시작 ---")

    # 1. PDF 문서 로드
    # os.walk를 사용하여 지정된 경로 내의 모든 하위 디렉토리까지 탐색하여 PDF 파일을 찾습니다.
    docs = []
    for root, _, files in os.walk(target["path"]):
        for file in files:
            if file.lower().endswith(".pdf"):
                full_path = os.path.join(root, file)
                try:
                    loader = PyMuPDFLoader(full_path)
                    docs.extend(loader.load())
                except Exception as e:
                    # PDF 로딩 중 오류 발생 시 메시지 출력
                    print(f"⚠️ 문서 로딩 실패: {full_path} → {e}")

    print(f"✔️ 총 로드된 PDF 문서 수: {len(docs)}개")
    if not docs:
        print("💡 로드할 PDF 문서가 없습니다. 다음 DB로 넘어갑니다.")
        continue # 문서가 없으면 현재 DB 처리를 건너뛰고 다음 DB로 이동

    # 2. 로드된 문서를 텍스트 청크로 분할
    chunks = splitter.split_documents(docs)
    # 내용이 비어있는 청크 제거 (간혹 발생할 수 있음)
    chunks = [doc for doc in chunks if doc.page_content.strip()]
    print(f"✔️ 유효한 텍스트 청크 수: {len(chunks)}개")
    
    if not chunks:
        print("💡 유효한 텍스트 청크가 없습니다. 다음 DB로 넘어갑니다.")
        continue # 청크가 없으면 현재 DB 처리를 건너뛰고 다음 DB로 이동

    # 3. Chroma DB 초기화 또는 로드
    # persist_directory 경로에 파일이 존재하고 비어있지 않으면 기존 DB를 로드합니다.
    # 그렇지 않으면 새로운 Chroma DB 인스턴스를 생성합니다.
    # 주의: Chroma.from_documents()는 기존 데이터를 덮어쓸 수 있습니다.
    # 여기서는 Chroma 객체를 먼저 생성하고 add_texts로 청크를 추가하여 기존 데이터에 병합하도록 합니다.
    if os.path.exists(target["persist_path"]) and len(os.listdir(target["persist_path"])) > 0:
        print(f"✨ 기존 {target['name']} DB 로드 중...")
        vectorstore = Chroma(
            collection_name=target["name"], # 컬렉션 이름 지정 (선택 사항이지만 유용)
            embedding_function=embedding_model, # 사용할 임베딩 모델 지정
            persist_directory=target["persist_path"] # DB가 저장된/저장될 디렉토리
        )
    else:
        print(f"✨ 새로운 {target['name']} DB 생성 중...")
        vectorstore = Chroma(
            collection_name=target["name"],
            embedding_function=embedding_model,
            persist_directory=target["persist_path"]
        )
        
    # 4. 분할된 청크를 Chroma DB에 배치(Batch)로 삽입
    print(f"🚀 {target['name']} DB에 청크를 삽입합니다 (총 {len(chunks)}개)...")
    # tqdm을 사용하여 청크 삽입 진행 상황을 시각적으로 표시합니다.
    for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc=f"{target['name']} 청크 삽입"):
        batch = chunks[i:i + BATCH_SIZE] # 현재 배치에 해당하는 청크를 추출
        texts = [doc.page_content for doc in batch] # 청크의 텍스트 내용만 추출
        metadatas = [doc.metadata for doc in batch] # 청크의 메타데이터만 추출

        try:
            # Chroma DB에 텍스트와 메타데이터를 추가
            vectorstore.add_texts(texts=texts, metadatas=metadatas)
        except Exception as e:
            # 벡터 삽입 중 오류 발생 시 메시지 출력
            print(f"❌ 벡터 삽입 실패 (배치 {i // BATCH_SIZE}): {e}")

    print(f"✅ {target['name']} 벡터 DB 생성/업데이트 완료 → {target['persist_path']}")

print("\n--- 모든 벡터 DB 처리 완료 ---")


--- DB1 DB 처리 시작 ---
✔️ 총 로드된 PDF 문서 수: 2172개
✔️ 유효한 텍스트 청크 수: 6067개
✨ 기존 DB1 DB 로드 중...
🚀 DB1 DB에 청크를 삽입합니다 (총 6067개)...


DB1 청크 삽입: 100%|██████████| 61/61 [03:11<00:00,  3.13s/it]


✅ DB1 벡터 DB 생성/업데이트 완료 → ./vector_db/db1_regulation

--- DB2 DB 처리 시작 ---
MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

✔️ 총 로드된 PDF 문서 수: 4794개
✔️ 유효한 텍스트 청크 수: 9378개
✨ 기존 DB2 DB 로드 중...
🚀 DB2 DB에 청크를 삽입합니다 (총 9378개)...


DB2 청크 삽입: 100%|██████████| 94/94 [05:00<00:00,  3.20s/it]

✅ DB2 벡터 DB 생성/업데이트 완료 → ./vector_db/db2_internal

--- 모든 벡터 DB 처리 완료 ---
